In [ ]:
pip install tensorflow==2.12

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras

In [ ]:
from google.colab import drive 
drive.mount('/content/drive') 
import pathlib 
InputPath = pathlib.Path("/content/drive/MyDrive/eyd") 
%cd /content/drive/MyDrive/eyd 
%ls

In [ ]:
he=['Elevation', 'Aspect', 'Slope','Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40',
       'target']
dataframe = pd.read_csv('q1-covtype.data.gz', names=he)

In [ ]:
dataframe.head()

In [ ]:
print(dataframe.shape)

In [ ]:
dataframe.head()

In [ ]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

In [ ]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [ ]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

In [ ]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [ ]:
from keras.utils import FeatureSpace

In [ ]:
feature_space = FeatureSpace(
    features={
        # Categorical features encoded as integers
        "Slope": FeatureSpace.integer_categorical(num_oov_indices=0),
        "Wilderness_Area1":"integer_categorical",
        "Wilderness_Area2":"integer_categorical",
        "Wilderness_Area3":"integer_categorical",
        "Wilderness_Area4":"integer_categorical",
        "Soil_Type1": "integer_categorical",
        "Soil_Type2": "integer_categorical",
        "Soil_Type3": "integer_categorical",
        "Soil_Type4": "integer_categorical",
        "Soil_Type5": "integer_categorical",
        "Soil_Type6": "integer_categorical",
        "Soil_Type7": "integer_categorical",
        "Soil_Type8": "integer_categorical",
        "Soil_Type9": "integer_categorical",
        "Soil_Type10": "integer_categorical",
        "Soil_Type11": "integer_categorical",
        "Soil_Type12": "integer_categorical",
        "Soil_Type13": "integer_categorical",
        "Soil_Type14": "integer_categorical",
        "Soil_Type15": "integer_categorical",
        "Soil_Type16": "integer_categorical",
        "Soil_Type17": "integer_categorical",
        "Soil_Type18": "integer_categorical",
        "Soil_Type19": "integer_categorical",
        "Soil_Type20": "integer_categorical",
        "Soil_Type21": "integer_categorical",
        "Soil_Type22": "integer_categorical",
        "Soil_Type23": "integer_categorical",
        "Soil_Type24": "integer_categorical",
        "Soil_Type25": "integer_categorical",
        "Soil_Type26": "integer_categorical",
        "Soil_Type27": "integer_categorical",
        "Soil_Type28": "integer_categorical",
        "Soil_Type29": "integer_categorical",
        "Soil_Type30": "integer_categorical",
        "Soil_Type31": "integer_categorical",
        "Soil_Type32": "integer_categorical",
        "Soil_Type33": "integer_categorical",
        "Soil_Type34": "integer_categorical",
        "Soil_Type35": "integer_categorical",
        "Soil_Type36": "integer_categorical",
        "Soil_Type37": "integer_categorical",
        "Soil_Type38": "integer_categorical",
        "Soil_Type39": "integer_categorical",
        "Soil_Type40": "integer_categorical",

        # Numerical features to normalize
        "Aspect": "float_normalized",
        "Horizontal_Distance_To_Hydrology": "float_normalized",
        "Vertical_Distance_To_Hydrology": "float_normalized",
        "Horizontal_Distance_To_Roadways": "float_normalized",
        "Hillshade_9am": "float_normalized",
        "Hillshade_Noon": "float_normalized",
        "Hillshade_3pm": "float_normalized",
        "Horizontal_Distance_To_Fire_Points": "float_normalized",
        "Elevation": "float_normalized",
    },

    output_mode="concat",
)

In [ ]:
train_ds_with_no_labels = train_ds.map(lambda x, _: x)
feature_space.adapt(train_ds_with_no_labels)

In [ ]:
for x, _ in train_ds.take(1):
    preprocessed_x = feature_space(x)
    print("preprocessed_x.shape:", preprocessed_x.shape)
    print("preprocessed_x.dtype:", preprocessed_x.dtype)

In [ ]:
preprocessed_train_ds = train_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_train_ds = preprocessed_train_ds.prefetch(tf.data.AUTOTUNE)

preprocessed_val_ds = val_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_val_ds = preprocessed_val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

x = keras.layers.Dense(32, activation="relu")(encoded_features)
x = keras.layers.Dropout(0.5)(x)
predictions = keras.layers.Dense(1, activation="sigmoid")(x)

training_model = keras.Model(inputs=encoded_features, outputs=predictions)
training_model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

inference_model = keras.Model(inputs=dict_inputs, outputs=predictions)

In [ ]:
training_model.fit(
    preprocessed_train_ds, epochs=20, validation_data=preprocessed_val_ds, verbose=2
)

In [ ]:
sample = dataframe[0:1].pop("target")

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = inference_model.predict(input_dict)

print(
    f"This particular patient had a {100 * predictions[0][0]:.2f}% probability "
    "of having a heart disease, as evaluated by our model."
)